# Load Images

In [1]:
import numpy as np
import nibabel as ni
import matplotlib.pyplot as plt
import joblib

In [2]:
def load(fname, affine=False, header=False):
    if header:
        affine = True
    
    data = ni.load(fname)
    if affine:
        affine_info = data.affine
    if header:
        header_info = data.header
        header_info.set_data_dtype(np.uint8)
    data = np.asarray(data.get_fdata(), dtype=np.float32, order='C')
    
    return data


mri_file = '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_ana.nii.gz'
mri_strip_file = '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_ana_strip.nii.gz'
seg_file = '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_seg_ana.nii.gz'
seg_TRI_file = '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_segTRI_ana.nii.gz'
seg_TRI_fill_file = '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_segTRI_fill_ana.nii.gz'
mri_mask_file =  '/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{subj}/IBSR_{subj}_ana_brainmask.nii.gz'



mri = load(mri_strip_file.format(subj='01'))
altura, largura, profundidade,_ = mri.shape
numero_de_pixels = altura * largura * profundidade
print("Número de pixels na imagem:", numero_de_pixels)

mri_mask = load(mri_mask_file.format(subj='01'))
mri_m = load(seg_TRI_fill_file.format(subj='01'))
print(mri_mask.shape)
#_, ax = plt.subplots(1, 3, figsize=(8, 8))

#ax[0].imshow(mri[:, :, 100])
#ax[1].imshow((mri_mask[:,:,100]))
#ax[2].imshow(mri_m[:,:,130])



Número de pixels na imagem: 8388608
(256, 128, 256, 1)


# Features 

Normalização -> Normalização por percentil, de modo a eliminar outliers. As intensidades estão entre 0 e 1.

In [3]:
def percentile_normalize(image, mask, lower_percentile=1, upper_percentile=99):
    
    brain = image[mask > 0]
    
    lower_bound = np.percentile(brain, lower_percentile)
    upper_bound = np.percentile(brain, upper_percentile)
    
    normalized_image = np.clip(image, lower_bound, upper_bound)
    
    normalized_image = (normalized_image - lower_bound) / (upper_bound - lower_bound)
    
    return normalized_image


Transformação Afim

In [4]:
def apply_affine_transform(image, lb_new, ub_new):

    transformed_image = image * (ub_new - lb_new) + lb_new
    
    return transformed_image



#normalized_image2 = percentile_normalize(mri[:,:,120], mri_mask[:,:,120])
#affim = apply_affine_transform(normalized_image2, 4, 7)

#_, ax = plt.subplots(1, 2, figsize=(8, 8))

#ax[0].imshow(normalized_image2)
#ax[1].imshow(affim)


Exponencial

In [5]:
def exponential_transformation(image):

    return np.exp(image)


Local Mean

In [6]:
from scipy.ndimage import convolve

def local_intensity_mean(img):
    
    kernel1 = np.ones((3, 3, 3, 1))/(3*3*3)
    kernel2 = np.ones((9, 9, 9, 1))/(9*9*9)

    convolved_image1 = convolve(img, kernel1, mode='constant')
    convolved_image2 = convolve(img, kernel2, mode='constant')

    return convolved_image1, convolved_image2

Gradiente

In [7]:
import numpy as np
from scipy.ndimage import sobel

def gradient_magnitude(image):
    # Compute the Sobel gradients along each axis
    gradient_x = sobel(image, axis=0)
    gradient_y = sobel(image, axis=1)
    gradient_z = sobel(image, axis=2)

    # Compute the gradient magnitude
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2 + gradient_z**2)

    return gradient_magnitude


# Cross-Validation

In [2]:
images = [] 
masks = []
segmentadas = []

sub = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17' ,'18']

for i in sub: 
    
    mri = load(mri_strip_file.format(subj=i))
    mri_mask = load(mri_mask_file.format(subj=i))
    mri_s = load(seg_TRI_fill_file.format(subj=i))

    images.append(mri)
    masks.append(mri_mask)
    segmentadas.append(mri_s)


NameError: name 'load' is not defined

Guardar imagens

NOTA: A numeração das imagens começa no 0 em vez de começar no 1.

In [44]:
for i, (ima, mas) in enumerate(zip(images, masks), start=1):
    exp = exponential_transformation(apply_affine_transform(percentile_normalize(ima, mas), 4, 7))
    filename = f"/Users/sofiaribas/Downloads/SEIM/exp_1/exp_{i:02d}.joblib"
    joblib.dump(exp, filename)


In [45]:
for i, (ima, mas) in enumerate(zip(images, masks), start=1):
    norm = percentile_normalize(ima,mas)
    filename = f"/Users/sofiaribas/Downloads/SEIM/norm_1/norm_{i:02d}.joblib" 
    joblib.dump(norm, filename)

In [46]:
for i, (ima, mas) in enumerate(zip(images, masks), start=1):
    local1, local2 = local_intensity_mean(ima)
    grad = gradient_magnitude(ima)

    filename_local1 = f"/Users/sofiaribas/Downloads/SEIM/local_1/local_{i:02d}_3.joblib"
    filename_local2 = f"/Users/sofiaribas/Downloads/SEIM/local_1/local_{i:02d}_9.joblib"
    filename_grad = f"/Users/sofiaribas/Downloads/SEIM/grad_1/grad_{i:02d}.joblib"
    
    joblib.dump(local1, filename_local1)
    joblib.dump(local2, filename_local2)
    joblib.dump(grad, filename_grad)

# SVM

Criar a Matriz das Features 

Para cada sujeito selecionamos 1 000 000 pontos e a cada ponto aplicamos as 5 features. Para cada sujeito são selecionados voxeis diferentes.

In [24]:
import joblib
import numpy as np
import nibabel as nib

def extract_features(person_id):

    norm_path = f"/Users/sofiaribas/Downloads/SEIM/norm/norm_{person_id}.joblib"
    exp_path = f"/Users/sofiaribas/Downloads/SEIM/exp/exp_{person_id}.joblib"
    grad_path = f"/Users/sofiaribas/Downloads/SEIM/grad/grad_{person_id}.joblib"
    local3_path = f"/Users/sofiaribas/Downloads/SEIM/local/local_{person_id}_3.joblib"
    local9_path = f"/Users/sofiaribas/Downloads/SEIM/local/local_{person_id}_9.joblib"
    image_segmented = f"/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{person_id}/IBSR_{person_id}_segTRI_fill_ana.nii.gz"
    mri_mask_path = f"/Users/sofiaribas/Downloads/SEIM/IBSR_nifti_stripped/IBSR_{person_id}/IBSR_{person_id}_ana_brainmask.nii.gz"

    imagem_norm = joblib.load(norm_path)
    imagem_exp = joblib.load(exp_path)
    imagem_grad = joblib.load(grad_path)
    imagem_local3 = joblib.load(local3_path)
    imagem_local9 = joblib.load(local9_path)
    
    segmented_data = nib.load(image_segmented).get_fdata()
    mask = nib.load(mri_mask_path).get_fdata()
    non_zero_coords = np.where(mask != 0)
    
    num_points = 800 
    selected_indices = np.random.choice(len(non_zero_coords[0]), num_points, replace=False)
    
    extracted_matrix = np.zeros((num_points, 5))
    pontos_seg = np.zeros((num_points, 1))

    for i, idx in enumerate(selected_indices):
        x, y, z = non_zero_coords[0][idx], non_zero_coords[1][idx], non_zero_coords[2][idx]

        extracted_matrix[i, 0] = imagem_norm[x, y, z]
        extracted_matrix[i, 1] = imagem_exp[x, y, z]
        extracted_matrix[i, 2] = imagem_grad[x, y, z]
        extracted_matrix[i, 3] = imagem_local3[x, y, z]
        extracted_matrix[i, 4] = imagem_local9[x, y, z]

        pontos_seg[i, 0] = segmented_data[x, y, z]
        
    return extracted_matrix, pontos_seg

sub = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18']


# Process data for each person
for person_id in sub:
    extracted_matrix, pontos_seg = extract_features(person_id)
print(extracted_matrix.size)


4000


NOTA: ver questão das labels

In [27]:
import joblib
import numpy as np
import nibabel as nib
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from medpy.metric.binary import dc
import random


def train_svm(X_train, y_train, C, gamma):

    svm = SVC(C=C, gamma=gamma, kernel='rbf')
    svm.fit(X_train, y_train)

    return svm


# Lista para armazenar os índices de treino e teste para cada fold
indices = []

# Dividir os dados em folds
for i in range(9):  
    test_start = i * 2  # Dois pacientes por fold
    test_end = test_start + 2
    
    # Índices para teste
    test_indices = list(range(test_start, test_end))
    
    # Índices para treino
    train_indices = [idx for idx in range(18) if idx not in test_indices]
    
    # Randomly shuffle the training indices
    random.shuffle(train_indices)
    
    # Split the shuffled training indices into training and validation
    train_fold_size = len(train_indices) - 2
    
    train_fold_indices = train_indices[:train_fold_size]
   
    val_fold_indices = train_indices[train_fold_size:]
   
    indices.append((train_fold_indices, val_fold_indices, test_indices))
#------------
def dice(predict, val):
    


#------------

# Loop de validação cruzada
for fold, (train_indices, val_indices, test_indices) in enumerate(indices):
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    
    # Coletar dados de treino e validação
    for idx in train_indices:
        person_id = sub[idx]
        extracted_matrix, pontos_seg = extract_features(person_id)
        X_train.append(extracted_matrix)
        y_train.append(pontos_seg)
        
    for idx in val_indices:
        person_id = sub[idx]
        extracted_matrix, pontos_seg = extract_features(person_id)
        X_val.append(extracted_matrix)
        y_val.append(pontos_seg)
    
    # Concatenar e transformar dados
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train).ravel()
    X_val = np.concatenate(X_val)
    y_val = np.concatenate(y_val).ravel()

    svm = train_svm(X_train, y_train, C=1, gamma='scale')
    y_pred = svm.predict(X_val)
    print(np.count_nonzero(y_pred==1), np.count_nonzero(y_pred==2), np.count_nonzero(y_pred==3))
    print(np.count_nonzero(y_val==1), np.count_nonzero(y_val==2), np.count_nonzero(y_val==3))
    dice = dc(y_pred==1, y_val==1)
                       
    print(f"Fold {fold + 1} - Dice: {dice:.4f}")


565 706 329
123 954 523
Fold 1 - Dice: 1.0000
106 1083 411
130 992 478
Fold 2 - Dice: 1.0000
573 589 438
126 949 525
Fold 3 - Dice: 1.0000
104 898 598
148 880 572
Fold 4 - Dice: 1.0000
503 666 431
115 902 583
Fold 5 - Dice: 1.0000
76 919 605
127 947 526
Fold 6 - Dice: 1.0000
25 1152 423
138 946 516
Fold 7 - Dice: 1.0000
47 1059 494
152 937 511
Fold 8 - Dice: 1.0000
47 1067 486
152 962 486
Fold 9 - Dice: 1.0000


In [ ]:
    best_dice = 0
    best_C = None
    best_gamma = None
    
    for C in [0.1, 1, 10]:
        for gamma in [0.001, 0.01, 0.1]:
            svm = train_svm(X_train, y_train, C, gamma)
            y_pred = svm.predict(X_val)
            
            dice = dice_coefficient(y_val, y_pred)
            
            if dice > best_dice:
                best_dice = dice
                best_C = C
                best_gamma = gamma
                
    print(f"Fold {fold + 1} - Melhores hiperparâmetros: C={best_C}, gamma={best_gamma}, Dice={best_dice:.4f}")